<a href="https://colab.research.google.com/github/rajgit-123/MyProject/blob/master/ANN_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df_train = pd.read_csv('/content/IMDB/train_mpst.csv')
df_train.head()
df_train.info()
df_train.describe()



In [11]:
def format_data(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text



In [12]:
df_train['plot_synopsis'] = df_train['plot_synopsis'].apply(lambda txt: format_data(txt))
df_train.columns
print(df_train.columns)

Index(['imdb_id', 'title', 'plot_synopsis', 'synopsis_source', 'absurd',
       'action', 'adult comedy', 'allegory', 'alternate history',
       'alternate reality', 'anti war', 'atmospheric', 'autobiographical',
       'avant garde', 'blaxploitation', 'bleak', 'boring', 'brainwashing',
       'christian film', 'claustrophobic', 'clever', 'comedy', 'comic',
       'cruelty', 'cult', 'cute', 'dark', 'depressing', 'dramatic',
       'entertaining', 'fantasy', 'feel-good', 'flashback', 'good versus evil',
       'gothic', 'grindhouse film', 'haunting', 'historical',
       'historical fiction', 'home movie', 'horror', 'humor', 'insanity',
       'inspiring', 'intrigue', 'magical realism', 'melodrama', 'murder',
       'mystery', 'neo noir', 'non fiction', 'paranormal', 'philosophical',
       'plot twist', 'pornographic', 'prank', 'psychedelic', 'psychological',
       'queer', 'realism', 'revenge', 'romantic', 'sadist', 'satire', 'sci-fi',
       'sentimental', 'storytelling', 'stupid',

In [13]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [20]:
df_val =  pd.read_csv('/content/IMDB/val.csv')
df_test = pd.read_csv('/content/IMDB/test.csv')
df_test_1 = df_test
df_test_1.rename(columns={'imdb_id':'ID'}, inplace=True)

X_train = df_train['plot_synopsis'].values
X_val = df_val['plot_synopsis'].values
X_test = df_test['plot_synopsis'].values
y_train = df_train.iloc[:, 4:].values
y_val = df_val.iloc[:, 4:].values

In [21]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(X_train)
xval_tfidf = tfidf_vectorizer.transform(X_val)
xtest_tfidf = tfidf_vectorizer.transform(X_test)

In [22]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [23]:
Logreg = LogisticRegression()
clf = OneVsRestClassifier(Logreg)

In [24]:
# fit model on train data
clf.fit(xtrain_tfidf, y_train)
# predict probabilities
# make predictions for validation set
y_pred = clf.predict(xval_tfidf)
y_pred[3]
y_pred_test = clf.predict(xtest_tfidf)
print(y_pred_test.shape)
print(df_test_1.shape)

(2966, 71)
(2966, 4)


In [25]:
df_test_submit_output = df_test_1
df_test_submit_output[['absurd',
       'action', 'adult comedy', 'allegory', 'alternate history',
       'alternate reality', 'anti war', 'atmospheric', 'autobiographical',
       'avant garde', 'blaxploitation', 'bleak', 'boring', 'brainwashing',
       'christian film', 'claustrophobic', 'clever', 'comedy', 'comic',
       'cruelty', 'cult', 'cute', 'dark', 'depressing', 'dramatic',
       'entertaining', 'fantasy', 'feel-good', 'flashback', 'good versus evil',
       'gothic', 'grindhouse film', 'haunting', 'historical',
       'historical fiction','home movie', 'horror', 'humor', 'insanity',
       'inspiring', 'intrigue', 'magical realism', 'melodrama', 'murder',
       'mystery', 'neo noir', 'non fiction', 'paranormal', 'philosophical',
       'plot twist', 'pornographic', 'prank', 'psychedelic', 'psychological',
       'queer', 'realism', 'revenge', 'romantic', 'sadist', 'satire', 'sci-fi',
       'sentimental', 'storytelling', 'stupid', 'suicidal', 'suspenseful',
       'thought-provoking', 'tragedy', 'violence', 'western', 'whimsical']] = y_pred_test

df_test_submit_output = df_test_submit_output.filter(['ID','absurd',
       'action', 'adult comedy', 'allegory', 'alternate history',
       'alternate reality', 'anti war', 'atmospheric', 'autobiographical',
       'avant garde', 'blaxploitation', 'bleak', 'boring', 'brainwashing',
       'christian film', 'claustrophobic', 'clever', 'comedy', 'comic',
       'cruelty', 'cult', 'cute', 'dark', 'depressing', 'dramatic',
       'entertaining', 'fantasy', 'feel-good', 'flashback', 'good versus evil',
       'gothic', 'grindhouse film', 'haunting', 'historical',
       'historical fiction', 'home movie', 'horror', 'humor', 'insanity',
       'inspiring', 'intrigue', 'magical realism', 'melodrama', 'murder',
       'mystery', 'neo noir', 'non fiction', 'paranormal', 'philosophical',
       'plot twist', 'pornographic', 'prank', 'psychedelic', 'psychological',
       'queer', 'realism', 'revenge', 'romantic', 'sadist', 'satire', 'sci-fi',
       'sentimental', 'storytelling', 'stupid', 'suicidal', 'suspenseful',
       'thought-provoking', 'tragedy', 'violence', 'western', 'whimsical'])
df_test_submit_output.to_csv('submission-imdb.csv', index=False)
